## Uso del modelo preentrenado de BART

## 1) Instalar librerias

In [1]:
pip install torch torchvision torchaudio transformers librosa sounddevice

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install sacremoses

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import math
import numpy as np
import re

## 2) Luego cargar el modelo preentrenado

In [7]:
## Luego para cargar
from transformers import BartTokenizer, BartForConditionalGeneration

model_path = "D:/Jose/UNI/Maestria IA/Cursos/03_Tercer_ciclo/03_Proyecto_Tesis/Proyecto/Modelo/Local/Bart/Quechua/bart_traducido_final"

tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

### 3) Definiendo las funciones de traducción del modelo

In [8]:
MAX_SEQ_LEN = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sentence_to_indices(sentence, word2idx):
    return [word2idx.get(word, word2idx['<unk>']) for word in sentence.split()]

def indices_to_sentence(indices, idx2word):
    filtered_indices = [idx for idx in indices if idx not in (spa_word2idx['<sos>'], spa_word2idx['<unk>'])]
    ## return ' '.join([idx2word[idx] for idx in indices if idx in idx2word and idx2word[idx] != '<pad>'])
    return ' '.join(spa_idx2word[idx] for idx in filtered_indices if idx in spa_idx2word)

def translate_sentence(model, sentence, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device='cpu'):
    model.eval()
    sentence = preprocess_sentence(sentence)
    input_indices = sentence_to_indices(sentence, eng_word2idx)
    input_tensor = torch.tensor(input_indices).unsqueeze(0).to(device)

    # Initialize the target tensor with <sos> token
    tgt_indices = [spa_word2idx['<sos>']]
    tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(device)

    with torch.no_grad():
        for _ in range(max_len):
            output = model(input_tensor, tgt_tensor)
            output = output.squeeze(0)
            next_token = output.argmax(dim=-1)[-1].item()
            tgt_indices.append(next_token)
            tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(device)
            if next_token == spa_word2idx['<eos>']:
                break

    return indices_to_sentence(tgt_indices, spa_idx2word)

### 4) Utilizando el modelo entrenado para traducir en del español al ingles un audio

In [9]:
import speech_recognition as sr

def audio_to_text(audio_data):
    recognizer = sr.Recognizer()
    
    # Convertir el audio grabado a un formato que SpeechRecognition pueda usar
    audio_data = sr.AudioData(audio_data.tobytes(), 44100, 1)
    
    try:
        # Reconocer el audio usando Google Web Speech API
        text = recognizer.recognize_google(audio_data, language='es-ES')  # Cambia 'es-ES' si usas otro idioma
        return text
    except sr.UnknownValueError:
        return "No se pudo entender el audio"
    except sr.RequestError as e:
        return f"No se pudo solicitar resultados; {e}"

# Asegúrate de integrar esta función en tu flujo principal

#### Probando grabacion audio

In [10]:
import sounddevice as sd
import numpy as np

def record_audio(duration=5, fs=44100):
    print("Grabando...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()  # Esperar a que termine la grabación
    return audio

def play_audio(audio):
    print("Reproduciendo...")
    sd.play(audio, 44100)
    sd.wait()  # Esperar a que termine de reproducir

# Graba audio
audio_data = record_audio(duration=5)
play_audio(audio_data)  # Reproduce el audio grabado

Grabando...
Reproduciendo...


## 5) Utilizando el modelo entrenado para la traducción de la voz en tiempo real

In [11]:
import torch
import sounddevice as sd
import numpy as np
import librosa
import time
from IPython.display import Audio, display


# Función para traducir texto
#def translate_text(text):
#    m_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#    translation = translate_sentence(model, text, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device=m_device)
#    return translation

def translate_text(text, model, tokenizer, device='cpu', max_length=50):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Función para generar voz
def text_to_speech(text):
    # Aquí puedes utilizar una biblioteca de TTS
    # Este es un ejemplo usando gTTS
    from gtts import gTTS
    import os
    
    tts = gTTS(text, lang='en')
    tts.save("output.mp3")
    ##os.system("start output.mp3")  # Cambia 'start' según tu sistema operativo
                                     # Se comenta para que no se ejecute doble.

    # Reproducir audio directamente en Jupyter
    display(Audio("output.mp3", autoplay=True))

# Captura de audio en tiempo real
##def record_audio(duration=5, fs=44100):
def record_audio(duration=5, fs=16000):
    print("Grabando...")
    ##audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float64')
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Esperar a que termine la grabación
    return audio.flatten()
    #return audio

# Procesamiento en tiempo real
#def real_time_translation():
#    while True:
#        audio_data = record_audio(duration=5)  # Graba 5 segundos de audio
#        # Supongamos que tienes una función para convertir audio a texto
#        text = audio_to_text(audio_data)  # Función ficticia que convierte audio a texto
#        print(f"Texto detectado: {text}")
#
#        translated_text = translate_text(text)
#        print(f"Texto traducido: {translated_text}")
#
#        text_to_speech(translated_text)
#        time.sleep(1)  # Esperar un segundo antes de la próxima grabación

def real_time_translation(model, tokenizer, device):
    while True:
        audio_data = record_audio(duration=5)
        
        # Aquí debes implementar o conectar con una función de STT como Whisper
        text = audio_to_text(audio_data)  # Asegúrate de definir esta función

        print(f"Texto detectado: {text}")
        
        translated_text = translate_text(text, model, tokenizer, device)
        print(f"Texto traducido: {translated_text}")
        
        text_to_speech(translated_text)
        time.sleep(1)

# Ejecutar
if __name__ == '__main__':
    # real_time_translation()
    try:
        #real_time_translation()
        real_time_translation(model, tokenizer, device)
    except KeyboardInterrupt:
        print("Detenido por el usuario.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")

Grabando...
Texto detectado: Hola
Texto traducido: Rola


Grabando...
Texto detectado: No se pudo entender el audio
Texto traducido: Manam chaymantaqa entienderqachu


Grabando...
Texto detectado: Ah por eso le pasó
Texto traducido: Chayraykum pasarqa


Grabando...
Texto detectado: No se pudo entender el audio
Texto traducido: Manam chaymantaqa entienderqachu


Grabando...
Texto detectado: No se pudo entender el audio
Texto traducido: Manam chaymantaqa entienderqachu


...

Grabando...
Texto detectado: mantenerse en el despacho
Texto traducido: kallpanchakuychik


Grabando...
Texto detectado: No se pudo entender el audio
Texto traducido: Manam chaymantaqa entienderqachu


Grabando...
Texto detectado: ah
Texto traducido: ah


Grabando...
Texto detectado: Buenos días
Texto traducido: Allin punchaw


Grabando...
Detenido por el usuario.
